In [1]:
import json, pandas as pd, os, numpy as np
from datetime import datetime
from constants import *
import copy

# Data calculation

In [2]:
# import result data
results = pd.read_csv(os.path.join('..', 'data', 'results.csv'), index_col=0)

In [3]:
# filter out warm-up task
results = results[results.task != 1]

In [4]:
# filters out paths from unneccessary interactions, especially the web variants (e.g. hovers for click variant)
def filterPath(path, web_variant):

    if(path):
        path = json.loads(path)
    else:
        return None

    # if variant is WTC or mobile, filter only click and changemind events
    if(web_variant == 'WTC' or web_variant == 'WM' or web_variant == 'WTCI'):
        clean_path = list(map(lambda x: x['node'], list(filter(lambda x: x['type'] == 'click', path))))

    # else do nothing
    else:
        return list(map(lambda x: x['node'], path))
    
    # format the path
    clean_path = list(map(lambda x: 'B' if x == 'back' else 'C' if x == 'changemind' else x, clean_path))
    return clean_path

In [5]:
# backtrack and stepbacks counting, for each variant separately
def backtracks(path, variant):
    whole_path = []
    backclick_count = 0
    last_backclick = False
    backtrack_count = 0
    backstep_count = 0
    active_nodes = [path[0]] if len(path) > 0 else []
    last_active_nodes = None
    
    # we iterate over interactions and calculate current tree positions
    for index, item in enumerate(path):
        backclick = False
        
        # we start with second interaction
        if(index != 0):

            # first variant
            if(variant == 'TP'):
                
                # if currently clicked node is active, it means we are closing a node
                # we remove all nodes which path starts with current node path
                # including current node 
                if(item in active_nodes):
                    active_nodes = list(filter(lambda x: not x.startswith(item), active_nodes))
                
                # otherwise we are opening a new node
                # we append it but before we filter out all nodes that are on the same or lower level 
                # as currently clicked node (for scenarios where we do not select direct descendant)
                else:
                    active_nodes = active_nodes[0:item.count('-')]
                    active_nodes.append(item)

            # second variant
            if(variant == 'TC'):

                # if currently clicked node is active, we are going back
                if(item in active_nodes):

                    # if we are closing the last opened node, we go one level back
                    # leaf nodes are not clickable
                    if(item == active_nodes[len(active_nodes)-1] and item not in LEAF_NODES):
                        active_nodes = active_nodes[0:len(active_nodes)-1]
                    
                    # otherwise we filter out all nodes which path starts with current node path
                    # EXcluding the currently clicked node
                    else:
                        active_nodes = list(filter(lambda x: not x.startswith(item), active_nodes))
                        active_nodes.append(item)
                
                # H always empties the active node array, since we are at the root
                elif(item == 'H'):
                    active_nodes = []
                
                # otherwise just append clicked node
                else:
                    active_nodes.append(item)
            
            # third variant
            if(variant == 'TO'):

                # if currently clicked node is active, we are going back
                if(item in active_nodes):

                    # if we are closing the last opened node, we go one level back
                    if(item == active_nodes[len(active_nodes)-1]):
                        active_nodes = active_nodes[0:len(active_nodes)-1]
                    
                    # otherwise the leaf node might me selected
                    # we filter out all nodes which path starts with current node path
                    # including the curent node
                    else:
                        active_nodes = list(filter(lambda x: not x.startswith(item), active_nodes))
                
                # otherwise, append the node
                # we append it but after we filter out all nodes that are on the same or lower level 
                # as currently clicked node (for scenarios where we do not select direct descendant)
                else:
                    active_nodes = active_nodes[0:item.count('-')]
                    active_nodes.append(item)
            
            if(variant == 'WTC' or variant == 'WTCI'):

                # if item is somewhere in the active  nodes
                if(item in active_nodes):

                    # filter out the item with all children
                    active_nodes = list(filter(lambda x: not x.startswith(item), active_nodes))

                    # solution for the special case of clicking on the same item multiple times
                    if((len(path) > index + 1) and path[index+1].startswith(item) and path[index+1] != item):
                        active_nodes.append(item)
                else:

                    # filter out all nodes on the same and lower level
                    active_nodes = active_nodes[0:item.count('-')]

                    # if changemind was not clicked, append current item
                    if(item != 'C'):
                        active_nodes.append(item)
            
            # if(variant == 'WTH' or variant == 'WLH'):
            #     active_nodes = active_nodes[0:item.count('-')]
            #     if(item != 'C'):
            #         active_nodes.append(item)
                
            if(variant == 'WM'):

                # if item is the back button, go one level up
                if(item == 'B'):
                    active_nodes = active_nodes[0:len(active_nodes)-1]
                else:

                    # else filter out same or lover level nodes
                    active_nodes = active_nodes[0:item.count('-')]

                    # if changemind was not clicked, append current item
                    if(item != 'C'):
                        active_nodes.append(item)

            # back step and back track calculation
            # if we are going up or stay at the same level
            if(len(last_active_nodes) >= len(active_nodes)):

                # if we have have moved in the structure
                if(last_active_nodes != active_nodes):

                    # stepback always true
                    backclick = True
                    backclick_count += 1

                    if(len(last_active_nodes) > len(active_nodes)):
                        backstep_count += len(last_active_nodes) - len(active_nodes)
                    else:
                        backstep_count += 1

                    # if previous move was stepback and we have moved up, it is not a backtrack
                    if(not last_backclick or (last_backclick and len(last_active_nodes) == len(active_nodes))):
                        backtrack_count += 1
                
        
        last_active_nodes = active_nodes.copy()
        last_backclick = backclick
        whole_path.append(copy.deepcopy(active_nodes))
        
    return (backclick_count, backtrack_count, backstep_count, whole_path)

In [6]:
# calculate variables
results['success'], results['directSuccess'], results['direct'] = [False, False, False]
results['firstClick'], results['finalClick'] = [None, None]
results['pathLength'], results['backclicks'], results['backtracks'], results['backsteps'] = [None, None, None, None]
results['path'] = None

for index, row in results.iterrows():

    # path from interactions
    path = filterPath(results.loc[index, 'interactions'], row.variant)
    results.loc[index, 'interactions'] = json.dumps(path)

    # direct_success
    if(path == CORRECT_PATHS[row.task]):
        results.loc[index, 'directSuccess'] = True

    # success
    if(len(path) > 0 and path[len(path)-1] == CORRECT_PATHS[row.task][len(CORRECT_PATHS[row.task])-1]):
        results.loc[index, 'success'] = True

    # stepbacks and backtracks
    counts = backtracks(path, row.variant)
    results.loc[index, 'backclicks'] = counts[0]
    results.loc[index, 'backtracks'] = counts[1]
    results.loc[index, 'backsteps'] = counts[2]
    results.loc[index, 'path'] = json.dumps(counts[3])

    # first click, last clicks and path lengths
    results.loc[index, 'firstClick'] = path[0] if len(path) > 0 else None
    results.loc[index, 'finalClick'] = list(filter(lambda x: x not in ['C', 'B'], path))[-1] if len(path) > 0 else None
    results.loc[index, 'pathLength'] = len(path)

    # direct
    results.loc[index, 'direct'] = False if results.loc[index, 'backclicks'] > 0 and len(path) > 0 and path[len(path)-1] in LEAF_NODES else True

In [7]:
# insert first and last click names
results['firstClickName'] = None
results['firstClickName'] = results['firstClick'].map(lambda y: find_by('path_num', str(int(y)))['name'] if not pd.isnull(y) else None)
results['finalClickName'] = None
results['finalClickName'] = results['finalClick'].map(lambda y: find_by('path_num', y)['name'] if (not pd.isnull(y) and type(y)==str) else None)

In [8]:
# export
results.to_csv(os.path.join('..', 'data', 'results.csv'))